In [1]:
import tensorflow as tf
from tensorflow import *

In [14]:
df = pd.read_csv("data_train.csv")

<IPython.core.display.Javascript object>

In [15]:
df.head()

,real_1,real_2,real_3,real_4,real_5,real_6,real_7,real_8,real_9,real_10,...,imag_993,imag_994,imag_995,imag_996,imag_997,imag_998,imag_999,imag_1000,SMF_length,Type_of_modulation
0,0.0292,0.0219,0.0091,0.0045,0.0178,0.0076,0.0322,0.0026,0.0177,0.0176,...,-0.0115,0.0036,-0.0011,0.0086,0.0007,-0.0127,-0.0015,-0.0134,10,OOK
1,0.0149,0.0359,0.0120,-0.0054,0.0153,0.0156,0.0133,-0.0044,0.0349,0.0107,...,0.0001,0.0011,-0.0006,0.0054,-0.0096,-0.0053,0.0029,-0.0065,10,OOK
2,0.0218,0.0271,0.0120,0.0110,0.0217,0.0124,0.0238,0.0007,0.0082,0.0021,...,0.0101,-0.0052,-0.0059,0.0027,-0.0032,-0.0043,0.0024,-0.0032,10,OOK
3,0.0128,0.0199,0.0007,-0.0007,0.0269,0.0034,0.0356,-0.0133,0.0272,0.0146,...,0.0092,-0.0060,-0.0181,-0.0073,0.0126,-0.0032,-0.0009,-0.0052,10,OOK
4,0.0288,0.0207,0.0069,0.0099,0.0353,0.0076,0.0265,-0.0080,0.0288,-0.0092,...,-0.0070,-0.0081,-0.0145,0.0057,0.0082,-0.0100,0.0022,-0.0065,10,OOK


In [16]:
modulation_type = pd.get_dummies(df['Type_of_modulation'])
modulation_type

<IPython.core.display.Javascript object>

,BPSK,OOK,QPSK
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
35995,0,0,1
35996,0,0,1
35997,0,0,1
35998,0,0,1


In [17]:
df.drop('Type_of_modulation', axis=1, inplace=True)

In [18]:
df = pd.concat([df, modulation_type], axis = 1)

<IPython.core.display.Javascript object>

In [19]:

output_categories = pd.concat([df['BPSK'], df['OOK'], df['QPSK']], axis = 1)

output_categories.shape

<IPython.core.display.Javascript object>

(36000, 3)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['SMF_length', 'BPSK', 'OOK', 'QPSK'], axis = 1),
                                                output_categories,
                                                test_size = 0.2,
                                                random_state = 42)

<IPython.core.display.Javascript object>

In [21]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                y_train,
                                                test_size = 0.25,
                                                random_state = 101)

<IPython.core.display.Javascript object>

In [22]:
x_train = np.array(x_train).reshape(-1, x_train.shape[1], 1)
x_test = np.array(x_test).reshape(-1,  x_test.shape[1], 1)
y_train = np.array(y_train).reshape(-1, y_train.shape[1])
y_test = np.array(y_test).reshape(-1, y_test.shape[1])
x_val = np.array(x_val).reshape(-1,  x_val.shape[1], 1)
y_val = np.array(y_val).reshape(-1, y_val.shape[1])
x_train.shape


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(21600, 2000, 1)

In [23]:
x_val.shape

(7200, 2000, 1)

In [24]:
model_mod = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu'),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu'),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(3, activation = 'softmax')
        ])

In [25]:
model_mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1998, 32)          128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 666, 32)           0         
_________________________________________________________________
dropout (Dropout)            (None, 666, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 664, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 221, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 221, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 219, 64)           1

In [26]:
model_mod.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model_mod.fit(x_train, y_train, batch_size=64, verbose=1, epochs=50,validation_data=(x_val, y_val))

Train on 21600 samples, validate on 7200 samples
Epoch 1/50
21600/21600 [==============================] - 91s 4ms/sample - loss: 0.0945 - accuracy: 0.9599 - val_loss: 2.7882e-04 - val_accuracy: 1.0000
Epoch 2/50
21600/21600 [==============================] - 65s 3ms/sample - loss: 1.0547e-04 - accuracy: 1.0000 - val_loss: 4.2196e-05 - val_accuracy: 1.0000
Epoch 3/50
21600/21600 [==============================] - 51s 2ms/sample - loss: 3.4747e-05 - accuracy: 1.0000 - val_loss: 1.7575e-05 - val_accuracy: 1.0000
Epoch 4/50
21600/21600 [==============================] - 52s 2ms/sample - loss: 1.2377e-05 - accuracy: 1.0000 - val_loss: 6.9013e-06 - val_accuracy: 1.0000
Epoch 5/50
21600/21600 [==============================] - 53s 2ms/sample - loss: 6.7526e-06 - accuracy: 1.0000 - val_loss: 4.0906e-06 - val_accuracy: 1.0000
Epoch 6/50
21600/21600 [==============================] - 50s 2ms/sample - loss: 3.2080e-06 - accuracy: 1.0000 - val_loss: 2.6098e-06 - val_accuracy: 1.0000
Epoch 7/50
21

In [28]:

model_mod.evaluate(x_test, y_test, verbose=1)

7200/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.5538519810201182e-07, 1.0]

In [29]:
for i in range(1, 10):
    x_val[i]
    y_val[i]
    testing_x = np.array(x_val[i]).reshape(-1, x_val.shape[1], 1)
    testing_y = np.array(y_val[i]).reshape(-1, y_val.shape[1])
    x = model_mod.predict(testing_x).round(1)
    print(f"Predicted {x}, Expected : {testing_y}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[0. 1. 0.]], Expected : [[0 1 0]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Predicted [[0. 0. 1.]], Expected : [[0 0 1]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[0. 1. 0.]], Expected : [[0 1 0]]


In [30]:
model_mod.save("model_mod.h5")

In [30]:
df = pd.read_csv("length_labelled.csv")

<IPython.core.display.Javascript object>

In [31]:
# modulation_type = pd.get_dummies(df['Type_of_modulation'])
df.drop('Type_of_modulation', axis=1, inplace=True)
# df = pd.concat([df, modulation_type], axis = 1)

In [32]:
df.head(10)

,real_1,real_2,real_3,real_4,real_5,real_6,real_7,real_8,real_9,real_10,...,imag_995,imag_996,imag_997,imag_998,imag_999,imag_1000,SMF_length,OSNR,Fm,Pulse Peak Power
0,0.033670,0.035331,0.035253,0.024534,0.006429,-0.001301,0.021491,0.000820,0.005125,0.000911,...,-0.019437,0.003776,0.006124,-0.006936,-0.008563,-0.020091,10,10,10000000000,0
1,0.021454,0.017926,0.014617,0.019884,0.002782,0.007080,0.017219,0.025417,-0.002450,0.002989,...,-0.001082,-0.010571,-0.001418,0.017246,0.009628,-0.008509,10,10,10000000000,0
2,0.027150,0.046041,0.019347,0.010905,-0.003669,0.029848,0.031202,0.015832,-0.023543,-0.003757,...,-0.000394,-0.003417,-0.000263,-0.002126,0.017930,-0.024731,10,10,10000000000,0
3,0.043701,0.038312,0.047442,0.036526,-0.000277,0.021282,0.046365,-0.017018,-0.005954,-0.024824,...,0.021770,-0.012912,0.000431,-0.006753,0.038043,0.000852,10,10,10000000000,0
4,0.015565,0.013733,0.049913,0.022555,0.004443,0.007014,0.048061,-0.001900,-0.009960,0.014544,...,0.005522,-0.001912,0.008499,0.010540,0.003414,0.018482,10,10,10000000000,0
5,0.013683,0.020358,0.010024,0.023190,-0.007839,0.039333,0.018679,-0.014142,-0.008312,-0.003339,...,-0.007880,0.001878,-0.000944,-0.016747,0.018830,-0.002094,10,10,10000000000,0
6,0.026280,0.007603,0.015116,0.051084,-0.001456,0.046676,0.019433,0.021965,-0.015515,0.010563,...,-0.000395,-0.019672,-0.007791,-0.008545,0.009967,0.011013,10,10,10000000000,0
7,0.033438,0.013458,0.031878,0.030113,0.000707,0.062561,0.037982,0.028631,-0.008637,0.007017,...,0.004390,-0.023915,-0.005729,-0.012468,-0.000267,0.012808,10,10,10000000000,0
8,0.018885,0.034873,0.023109,0.027182,-0.015413,0.021928,0.017390,0.020095,-0.000561,0.023018,...,0.008736,0.020442,-0.023876,0.007923,0.004274,-0.017045,10,10,10000000000,0
9,0.020008,0.038175,0.013427,0.029058,-0.005828,0.006615,0.016881,0.009163,0.000561,-0.013327,...,-0.021684,-0.005391,0.018362,0.006027,0.007213,0.008714,10,10,10000000000,0


In [3]:
pd.set_option('display.max_columns', 3000)

<IPython.core.display.Javascript object>

,real_1,real_2,real_3,real_4,real_5,real_6,real_7,real_8,real_9,real_10,...,real_992,real_993,real_994,real_995,real_996,real_997,real_998,real_999,real_1000,SMF_length
0,0.002075,0.020679,0.011420,0.007832,0.038306,0.013302,0.024427,0.037443,0.015739,0.011038,...,0.027366,0.044122,0.011171,0.015477,0.014624,0.001688,0.024925,0.046979,0.015337,10
1,0.021257,0.014637,0.016862,0.016974,0.021168,0.017448,0.034990,0.035818,0.008299,0.020064,...,0.011919,0.012571,0.021295,0.018518,0.002637,0.007267,0.015076,0.020230,0.018563,10
2,0.015333,0.013629,0.034417,0.004657,0.014902,0.026258,0.052499,0.042435,0.013535,0.020196,...,0.016692,0.016604,0.025551,0.011627,0.016686,0.010046,0.010612,0.018253,0.033461,10
3,0.008837,0.036134,0.035075,0.011128,0.032040,0.040992,0.019351,0.031148,0.009661,0.009209,...,0.008258,0.029049,0.011193,0.006242,0.002744,0.013935,0.033773,0.044531,0.036463,10
4,0.026234,0.037946,0.024602,0.012678,0.041692,0.022311,0.020961,0.054172,0.009380,0.032009,...,0.018271,0.033802,0.008542,0.031568,0.000887,0.010297,0.033996,0.031412,0.030235,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.045702,0.090348,0.020076,0.028588,0.042530,0.012112,0.053615,0.042145,0.024316,0.027485,...,0.020993,0.006624,0.011100,0.047269,0.017843,0.049848,0.105288,0.053609,0.029246,15
496,0.024611,0.011093,0.009618,0.029174,0.031186,0.008547,0.026667,0.040451,0.001898,0.055803,...,0.010103,0.017532,0.046680,0.058875,0.046992,0.021128,0.013434,0.030240,0.024470,15
497,0.139439,0.015221,0.017338,0.032799,0.036712,0.024205,0.017694,0.005517,0.000101,0.036591,...,0.030622,0.036735,0.010825,0.014971,0.063725,0.012014,0.023848,0.021105,0.043212,15
498,0.122446,0.069061,0.002989,0.022068,0.002304,0.061370,0.010942,0.057024,0.017350,0.002738,...,0.014466,0.051029,0.003552,0.061738,0.046969,0.020001,0.068868,0.002119,0.081465,15


In [43]:

# for j in range (0, 1000):
# #         print(df.iloc[i][j])
# #     x = df.iloc[i][j]
# #     y = df.iloc[i][1000+j]
# #         print(x, y, (x*x+y*y)**(1/2))
#     df.iloc[:, j] = (df.iloc[:, j]*df.iloc[:, j]+df.iloc[:, 1000+j]*df.iloc[:, 1000+j])**(1/2)
# #         print(df.iloc[i, j])

        
        
    

In [45]:
# To make a dataset of 129 samples

# df.drop(df.iloc[:, 129:2000],axis  = 1, inplace = True)
# df.drop(df.iloc[:, 257:1000],axis  = 1, inplace = True)
# df.drop('real_129', axis=1, inplace=True)

In [46]:
# x = np.array(df.iloc[:, 0:1000])
# x = x.flatten()
# y = np.array(df.iloc[:, 1000:2000])
# y = y.flatten()
# z = list()
# for i in range(0, 69600):
#     for j in range(0, 1000):
#         z.append(df['SMF_length'][i])
# z = np.array(z)
# z.shape
# data = {'modulated':x[0:69600000:1000], 'original' :y[0:69600000:1000], 'SMF_length' :z[0:69600000:1000]
# }
# x.shape   
# df1 = pd.DataFrame(data)
# df1.head()

In [47]:
# # to make a dataset of all 1000 samples
# df.drop(df.iloc[:, 1000:2000],axis  = 1, inplace = True)

In [74]:
df.shape

(34800, 2001)

In [5]:
df.shape

(69600, 2001)

In [6]:
for i in range(1, 10000, 2532):
    print(df['real_2'][i], df['ori_2'][i])

0.029948789624958134 0
0.012515989773086265 0
0.09040271013636704 1
0.05300764095863916 0


In [33]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['SMF_length'], axis = 1),
                                                df['SMF_length'],
                                                test_size = 0.2,
                                                random_state = 101)

<IPython.core.display.Javascript object>

In [34]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                y_train,
                                                test_size = 0.25,
                                                random_state = 101)

<IPython.core.display.Javascript object>

In [35]:
x_train = np.array(x_train).reshape(-1, x_train.shape[1], 1)
x_test = np.array(x_test).reshape(-1,  x_test.shape[1], 1)
y_train = np.array(y_train).reshape(y_train.shape[0], 1)
y_test = np.array(y_test).reshape(y_test.shape[0], 1)
x_val = np.array(x_val).reshape(-1,  x_val.shape[1], 1)
y_val = np.array(y_val).reshape(y_val.shape[0], 1)
x_train.shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(17280, 2003, 1)

In [46]:
model_length = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv1D(128,kernel_size = 3, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
#         tf.keras.layers.MaxPooling1D(pool_size=3),
#         tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(1)
        ])

In [47]:
# model_length = tf.keras.models.load_model('model_lenght.h5')

In [48]:
model_length.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 1998, 32)          128       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 666, 32)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 666, 32)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 664, 128)          12416     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 221, 128)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 221, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 221, 128)         

In [49]:
model_length.compile(loss='mae', optimizer='adam')

In [92]:
model_length.fit(x_train, y_train, batch_size=12, verbose=1, epochs=3,validation_data=(x_val, y_val))

Train on 20880 samples, validate on 6960 samples
Epoch 1/3
20880/20880 [==============================] - 149s 7ms/sample - loss: 5.3495 - val_loss: 5.0294
Epoch 2/3
20880/20880 [==============================] - 151s 7ms/sample - loss: 4.5047 - val_loss: 5.0942
Epoch 3/3
20880/20880 [==============================] - 147s 7ms/sample - loss: 4.4772 - val_loss: 7.7218


In [50]:
from tensorflow.keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 10,  
                                        restore_best_weights = True) 

In [51]:
checkpoint_filepath = 'weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [52]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, min_lr = 0.001)

In [53]:
model_length.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks = [model_checkpoint_callback])

Train on 21600 samples, validate on 7200 samples
Epoch 1/50
21600/21600 [==============================] - 274s 13ms/sample - loss: 10.0111 - val_loss: 18.5056
Epoch 2/50
21600/21600 [==============================] - 211s 10ms/sample - loss: 4.1519 - val_loss: 9.1075
Epoch 3/50
21600/21600 [==============================] - 207s 10ms/sample - loss: 3.5642 - val_loss: 5.8447
Epoch 4/50
21600/21600 [==============================] - 179s 8ms/sample - loss: 3.1443 - val_loss: 5.0171
Epoch 5/50
21600/21600 [==============================] - 190s 9ms/sample - loss: 2.9306 - val_loss: 3.1471
Epoch 6/50
21600/21600 [==============================] - 186s 9ms/sample - loss: 2.8807 - val_loss: 3.2671
Epoch 7/50
21600/21600 [==============================] - 185s 9ms/sample - loss: 2.7204 - val_loss: 4.9251
Epoch 8/50
21600/21600 [==============================] - 185s 9ms/sample - loss: 2.7474 - val_loss: 5.4552
Epoch 9/50
21600/21600 [==============================] - 176s 8ms/sample - loss: 

In [54]:
model_l = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv1D(128,kernel_size = 3, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(1)
        ])
model_l.load_weights("weights.05-3.15.h5")

In [57]:
model_l.compile(loss='mae', optimizer='adam')

In [58]:
model_l.evaluate(x_test, y_test, verbose=1)

7200/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

3.128783941268921

In [61]:
model_l.save("model_length.h5")

In [15]:
df = pd.read_csv("length_labelled.csv")

<IPython.core.display.Javascript object>

In [16]:
# modulation_type = pd.get_dummies(df['Type_of_modulation'])
df.drop('Type_of_modulation', axis=1, inplace=True)
# df = pd.concat([df, modulation_type], axis = 1)

In [17]:
pd.set_option('display.max_columns', 3000)

<IPython.core.display.Javascript object>

In [39]:
df.head()

real_1    real_2    real_3    real_4    real_5    real_6    real_7  \
0  0.033670  0.035331  0.035253  0.024534  0.006429 -0.001301  0.021491   
1  0.021454  0.017926  0.014617  0.019884  0.002782  0.007080  0.017219   
2  0.027150  0.046041  0.019347  0.010905 -0.003669  0.029848  0.031202   
3  0.043701  0.038312  0.047442  0.036526 -0.000277  0.021282  0.046365   
4  0.015565  0.013733  0.049913  0.022555  0.004443  0.007014  0.048061   

     real_8    real_9   real_10   real_11   real_12   real_13   real_14  \
0  0.000820  0.005125  0.000911  0.004824  0.010941  0.003991  0.003977   
1  0.025417 -0.002450  0.002989  0.012531  0.006979  0.017612  0.021448   
2  0.015832 -0.023543 -0.003757  0.027618  0.002890  0.014666  0.034077   
3 -0.017018 -0.005954 -0.024824  0.017967  0.006698 -0.007693  0.027936   
4 -0.001900 -0.009960  0.014544  0.001193  0.027967 -0.004185  0.019953   

    real_15   real_16   real_17   real_18   real_19   real_20   real_21  \
0 -0.007135 -0.004711  0.008247 -0.004437  0.001742  0.018987  0.023167   
1  0.007972  0.019105  0.026932 -0.009937 -0.000584  0.020350  0.024708   
2  0.004986  0.000664  0.031670  0.005011  0.027952  0.024376  0.031388   
3 -0.007496 -0.003800  0.032085  0.025493  0.023956  0.024737  0.017271   
4 -0.010274 -0.011286  0.027765  0.007851  0.017812  0.027243  0.015391   

    real_22   real_23   real_24   real_25   real_26   real_27   real_28  \
0  0.001037  0.010228  0.031836  0.038914  0.008298 -0.004868  0.007619   
1  0.017363  0.001477  0.024023  0.010146  0.005029 -0.003653  0.018659   
2  0.014931  0.008174  0.010318  0.025804  0.000376  0.014640  0.038689   
3 -0.010652  0.042217  0.030648  0.030936 -0.008973  0.014170  0.026120   
4 -0.010766  0.017926 -0.014464  0.027849  0.010385  0.003189  0.025238   

    real_29   real_30   real_31   real_32   real_33   real_34   real_35  \
0  0.050815  0.007221 -0.015312  0.017117  0.004013 -0.003650  0.013429   
1  0.035340  0.003192  0.008773  0.010276  0.017197  0.006196  0.006898   
2  0.020558 -0.004015 -0.003236  0.027252  0.013521  0.032323 -0.013008   
3  0.033319  0.019985  0.001840  0.012011  0.010723  0.030666 -0.018325   
4  0.011016 -0.002146  0.005870  0.013442  0.004304  0.018862 -0.014257   

    real_36   real_37   real_38   real_39   real_40   real_41   real_42  \
0  0.019609  0.025679  0.013074  0.018614  0.000883  0.025580  0.023014   
1  0.024692  0.019749  0.016782  0.045180  0.006810  0.027832  0.028945   
2  0.024935  0.030617  0.000257  0.043202  0.010714  0.023716  0.037717   
3  0.029701 -0.019479  0.039295  0.019658  0.007837  0.034473  0.030929   
4  0.016590 -0.007059  0.056977  0.023351  0.016312  0.044453  0.001989   

    real_43   real_44   real_45   real_46   real_47   real_48   real_49  \
0  0.029014  0.036334 -0.011489  0.009346 -0.002129 -0.023030 -0.008518   
1  0.021834  0.007998  0.019132  0.009537  0.002268 -0.009531  0.002856   
2  0.010693  0.026109  0.012480  0.021375  0.005247 -0.002902  0.019846   
3  0.023678  0.005602  0.004679  0.002271  0.011281  0.007388  0.010869   
4  0.038807  0.026616 -0.010918  0.003212  0.013693 -0.013759 -0.005384   

    real_50   real_51   real_52   real_53   real_54   real_55   real_56  \
0 -0.016820  0.030171  0.015953 -0.007915  0.001041 -0.011581  0.017106   
1 -0.000043  0.019001  0.025749 -0.016424  0.001292  0.012425 -0.010516   
2 -0.014981  0.019718  0.010810 -0.022736  0.007579  0.006033 -0.000954   
3 -0.002830  0.005349 -0.024125 -0.016679  0.011256  0.012959 -0.005934   
4  0.009246  0.027020  0.002396 -0.013591  0.025500 -0.002859 -0.011713   

    real_57   real_58   real_59   real_60   real_61   real_62   real_63  \
0  0.006854  0.029135 -0.014543 -0.002967  0.018316 -0.004898  0.011561   
1 -0.009097  0.019112 -0.001636 -0.011982  0.005563  0.003483  0.022504   
2  0.021975  0.021549 -0.006331 -0.005321 -0.023357 -0.010008  0.041123   
3  0.016517  0.035564 -0.032279 -0.009789 -0.013874  0.023631 -0.000697   
4  0.015056  0.014588 

In [18]:
df['Fm']/=10000000000

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['SMF_length'], axis = 1),
                                                df['SMF_length'],
                                                test_size = 0.2,
                                                random_state = 101)

<IPython.core.display.Javascript object>

In [20]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                y_train,
                                                test_size = 0.25,
                                                random_state = 101)

<IPython.core.display.Javascript object>

In [21]:
x_train = np.array(x_train).reshape(-1, x_train.shape[1], 1)
x_test = np.array(x_test).reshape(-1,  x_test.shape[1], 1)
y_train = np.array(y_train).reshape(y_train.shape[0], 1)
y_test = np.array(y_test).reshape(y_test.shape[0], 1)
x_val = np.array(x_val).reshape(-1,  x_val.shape[1], 1)
y_val = np.array(y_val).reshape(y_val.shape[0], 1)
x_train.shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(17280, 2003, 1)

In [23]:
model_revised = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(1)
        ])

In [8]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.01, patience = 5, min_lr = 0.0000001,cooldown =0, verbose = 1)

In [9]:
checkpoint_filepath = 'weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [10]:
from tensorflow.keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 10,  
                                        restore_best_weights = True) 

In [27]:
model_revised.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 2001, 32)          128       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 667, 32)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 667, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 665, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 221, 32)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 221, 32)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 221, 32)          

In [28]:
model_revised.compile(loss='mae', optimizer='RMSProp')

In [29]:
model_revised.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks = [model_checkpoint_callback, reduce_lr])

Train on 17280 samples, validate on 5760 samples
Epoch 1/50
17280/17280 [==============================] - 97s 6ms/sample - loss: 15.0980 - val_loss: 35.9906
Epoch 2/50
17280/17280 [==============================] - 82s 5ms/sample - loss: 11.8708 - val_loss: 29.1881
Epoch 3/50
17280/17280 [==============================] - 80s 5ms/sample - loss: 11.2530 - val_loss: 24.8757
Epoch 4/50
17280/17280 [==============================] - 80s 5ms/sample - loss: 11.0097 - val_loss: 24.5469
Epoch 5/50
17280/17280 [==============================] - 81s 5ms/sample - loss: 10.8068 - val_loss: 22.8969
Epoch 6/50
17280/17280 [==============================] - 80s 5ms/sample - loss: 10.7647 - val_loss: 23.0684
Epoch 7/50
17280/17280 [==============================] - 80s 5ms/sample - loss: 10.6944 - val_loss: 21.7812
Epoch 8/50
17280/17280 [==============================] - 80s 5ms/sample - loss: 10.6162 - val_loss: 19.8906
Epoch 9/50
17280/17280 [==============================] - 80s 5ms/sample - loss

In [17]:
model_revised_7 = tf.keras.Sequential([
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.BatchNormalization(),
# #         tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
# #         tf.keras.layers.MaxPooling1D(pool_size=3),
# #         tf.keras.layers.Dropout(0.4),
# #         tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
# #         tf.keras.layers.MaxPooling1D(pool_size=3),
# #         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1)
        ])
model_revised_7.compile(loss='mae', optimizer='RMSProp')
model_revised_7.fit(x_train, y_train, batch_size=8, verbose=1, epochs=5,validation_data=(x_val, y_val), callbacks = [reduce_lr])

Train on 21600 samples, validate on 7200 samples
Epoch 1/5
21600/21600 [==============================] - 126s 6ms/sample - loss: 14.3873 - val_loss: 11.8718
Epoch 2/5
21600/21600 [==============================] - 122s 6ms/sample - loss: 12.5935 - val_loss: 12.6595
Epoch 3/5
21600/21600 [==============================] - 124s 6ms/sample - loss: 12.4783 - val_loss: 12.3272
Epoch 4/5
21600/21600 [==============================] - 125s 6ms/sample - loss: 12.4079 - val_loss: 11.1360
Epoch 5/5
21600/21600 [==============================] - 125s 6ms/sample - loss: 12.3619 - val_loss: 12.8355


In [50]:
model_revised_7 = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv1D(64,kernel_size = 3, activation = 'relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),


        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1)
        ])
model_revised_7.summary()
model_revised_7.compile(loss='mae', optimizer='RMSProp')
model_revised_7.fit(x_train, y_train, batch_size=20, verbose=1, epochs=5,validation_data=(x_val, y_val), callbacks = [reduce_lr])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 2001, 32)          128       
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 667, 32)           0         
_________________________________________________________________
dropout_66 (Dropout)         (None, 667, 32)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 665, 64)           6208      
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 332, 64)           0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 332, 64)           0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 332, 64)         

In [29]:
model_revised_7.fit(x_train, y_train, batch_size=8, verbose=1, epochs=100,validation_data=(x_val, y_val), callbacks=[reduce_lr])

Train on 21600 samples, validate on 7200 samples
Epoch 1/100
21600/21600 [==============================] - 174s 8ms/sample - loss: 3.5535 - val_loss: 7.4754
Epoch 2/100
21600/21600 [==============================] - 176s 8ms/sample - loss: 3.5257 - val_loss: 6.1980
Epoch 3/100
21600/21600 [==============================] - 173s 8ms/sample - loss: 3.5789 - val_loss: 5.3861
Epoch 4/100
21600/21600 [==============================] - 173s 8ms/sample - loss: 3.5374 - val_loss: 7.1223
Epoch 5/100
21600/21600 [==============================] - 176s 8ms/sample - loss: 3.5488 - val_loss: 7.7227
Epoch 6/100
21600/21600 [==============================] - 175s 8ms/sample - loss: 3.5286 - val_loss: 6.0008
Epoch 7/100
21592/21600 [============================>.] - ETA: 0s - loss: 3.5387
Epoch 00007: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
21600/21600 [==============================] - 177s 8ms/sample - loss: 3.5381 - val_loss: 7.3486
Epoch 8/100
21600/21600 [============

KeyboardInterrupt: 

model_revised_7.save("higher_loss_6_with_DD.h5")

In [79]:
model_revised_6.compile(loss='mae', optimizer='RMSProp')

In [78]:
model_revised_6.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks=[reduce_lr])

Train on 17280 samples, validate on 5760 samples
Epoch 1/50
17280/17280 [==============================] - 110s 6ms/sample - loss: 45.2434 - val_loss: 76.0510
Epoch 2/50
17280/17280 [==============================] - 102s 6ms/sample - loss: 45.1517 - val_loss: 86.6287
Epoch 3/50
17280/17280 [==============================] - 102s 6ms/sample - loss: 46.2011 - val_loss: 81.2184
Epoch 4/50
17280/17280 [==============================] - 102s 6ms/sample - loss: 46.7859 - val_loss: 105.9282
Epoch 5/50
17280/17280 [==============================] - 101s 6ms/sample - loss: 45.4830 - val_loss: 71.8590
Epoch 6/50
17280/17280 [==============================] - 103s 6ms/sample - loss: 46.2289 - val_loss: 76.2487
Epoch 7/50
17264/17280 [============================>.] - ETA: 0s - loss: 46.4481
Epoch 00007: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
17280/17280 [==============================] - 101s 6ms/sample - loss: 46.4844 - val_loss: 123.9206
Epoch 8/50
17280/17280 [===

In [64]:
model_revised_4.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val))

Train on 17280 samples, validate on 5760 samples
Epoch 1/50
17280/17280 [==============================] - 57s 3ms/sample - loss: 2.2547 - val_loss: 3.6987
Epoch 2/50
17280/17280 [==============================] - 52s 3ms/sample - loss: 2.2765 - val_loss: 3.6850
Epoch 3/50
17280/17280 [==============================] - 50s 3ms/sample - loss: 2.2535 - val_loss: 3.6501
Epoch 4/50
17280/17280 [==============================] - 50s 3ms/sample - loss: 2.2777 - val_loss: 3.6742
Epoch 5/50
17280/17280 [==============================] - 49s 3ms/sample - loss: 2.2573 - val_loss: 3.6564
Epoch 6/50
17280/17280 [==============================] - 49s 3ms/sample - loss: 2.2626 - val_loss: 3.6805
Epoch 7/50
17280/17280 [==============================] - 50s 3ms/sample - loss: 2.2738 - val_loss: 3.6554
Epoch 8/50
17280/17280 [==============================] - 51s 3ms/sample - loss: 2.2801 - val_loss: 3.6951
Epoch 9/50
 4912/17280 [=======>......................] - ETA: 33s - loss: 2.2982

KeyboardInterrupt: 

In [74]:
model_revised_3 = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu'),
#         tf.keras.layers.MaxPooling1D(pool_size=3),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu'),
#         tf.keras.layers.MaxPooling1D(pool_size=3),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu'),
#         tf.keras.layers.MaxPooling1D(pool_size=3),
#         tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1)
        ])
model_revised_3.compile(loss='mae', optimizer='RMSProp')
model_revised_3.fit(x_train, y_train, batch_size=8, verbose=1, epochs=100,validation_data=(x_val, y_val), callbacks = [reduce_lr])

Train on 17280 samples, validate on 5760 samples
Epoch 1/100
17280/17280 [==============================] - 211s 12ms/sample - loss: 13.2943 - val_loss: 6.2519
Epoch 2/100
17280/17280 [==============================] - 198s 11ms/sample - loss: 9.3406 - val_loss: 4.3933
Epoch 3/100
17280/17280 [==============================] - 197s 11ms/sample - loss: 8.5502 - val_loss: 4.4273
Epoch 4/100
17280/17280 [==============================] - 197s 11ms/sample - loss: 8.0883 - val_loss: 5.8261
Epoch 5/100
17280/17280 [==============================] - 196s 11ms/sample - loss: 7.6723 - val_loss: 4.1750
Epoch 6/100
17280/17280 [==============================] - 196s 11ms/sample - loss: 7.4461 - val_loss: 3.6152
Epoch 7/100
17280/17280 [==============================] - 199s 11ms/sample - loss: 7.3268 - val_loss: 6.1261
Epoch 8/100
17280/17280 [==============================] - 194s 11ms/sample - loss: 7.1610 - val_loss: 6.0439
Epoch 9/100
17280/17280 [==============================] - 194s 11ms/s

In [75]:
model_revised_3.compile(loss='mae', optimizer='adam', lr = 0.01)

In [77]:
model_revised_3.save("to_be_trained.h5")

In [76]:
model_revised_3.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks = [reduce_lr])

KeyboardInterrupt: 

In [80]:
model_revised_6.evaluate(x_test, y_test, verbose=1)

5760/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

5.96738672653834

In [42]:
model_revised_2.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks = [reduce_lr, model_checkpoint_callback])

Train on 17280 samples, validate on 5760 samples
Epoch 1/50
17280/17280 [==============================] - 186s 11ms/sample - loss: 1.6015 - val_loss: 2.6298
Epoch 2/50
17280/17280 [==============================] - 188s 11ms/sample - loss: 1.5360 - val_loss: 2.6467
Epoch 3/50
17280/17280 [==============================] - 191s 11ms/sample - loss: 1.5321 - val_loss: 2.6906
Epoch 4/50
17280/17280 [==============================] - 195s 11ms/sample - loss: 1.5307 - val_loss: 2.7258
Epoch 5/50
17280/17280 [==============================] - 196s 11ms/sample - loss: 1.5093 - val_loss: 2.4993
Epoch 6/50
17280/17280 [==============================] - 197s 11ms/sample - loss: 1.5097 - val_loss: 2.5309
Epoch 7/50
17280/17280 [==============================] - 201s 12ms/sample - loss: 1.4905 - val_loss: 2.5211
Epoch 8/50
17280/17280 [==============================] - 222s 13ms/sample - loss: 1.5074 - val_loss: 2.6983
Epoch 9/50
17280/17280 [==============================] - 223s 13ms/sample - lo

In [43]:
model_revised_2.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val), callbacks = [reduce_lr, model_checkpoint_callback])

Train on 17280 samples, validate on 5760 samples
Epoch 1/50
17280/17280 [==============================] - 230s 13ms/sample - loss: 1.3190 - val_loss: 2.7610
Epoch 2/50
17280/17280 [==============================] - 227s 13ms/sample - loss: 1.3371 - val_loss: 2.6027
Epoch 3/50
17280/17280 [==============================] - 228s 13ms/sample - loss: 1.3380 - val_loss: 2.7566
Epoch 4/50
17280/17280 [==============================] - 226s 13ms/sample - loss: 1.3386 - val_loss: 2.8642
Epoch 5/50
17280/17280 [==============================] - 228s 13ms/sample - loss: 1.3363 - val_loss: 2.8287
Epoch 6/50
17280/17280 [==============================] - 221s 13ms/sample - loss: 1.3337 - val_loss: 2.9990
Epoch 7/50
17280/17280 [==============================] - 306s 18ms/sample - loss: 1.3222 - val_loss: 2.6267
Epoch 8/50
17280/17280 [==============================] - 313s 18ms/sample - loss: 1.3442 - val_loss: 2.6761
Epoch 9/50
17280/17280 [==============================] - 316s 18ms/sample - lo

In [45]:
model_revised_2.load_weights("weights.12-2.44.h5")
model_revised_2.evaluate(x_test, y_test, verbose=0)

2.465774999724494

In [46]:
for i in range(1, 100):
    if y_val[i]>=40:
        testing_x = np.array(x_test[i]).reshape(-1, x_test.shape[1], 1)
        testing_y = np.array(y_test[i]).reshape(-1, y_test.shape[1])
        x = model_revised_2.predict(testing_x).round(1)
        print(f"Predicted {x}, Expected : {testing_y}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[79.3]], Expected : [[80]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Predicted [[33.6]], Expected : [[40]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[39.9]], Expected : [[50]]


In [63]:
model_revised_4.save("model_length_new3_higherloss.h5")

In [61]:
path = r"C:\Users\Yuvi\Desktop\Design Lab Documents\Codes Design Lab\logs"

In [63]:
tb = tf.keras.callbacks.TensorBoard(log_dir=path,
                                 histogram_freq=0, 
                                 write_graph=True, 
                                 write_images=False,    
                                 update_freq='epoch', 
                                 profile_batch=2, 
                                 embeddings_freq=0,    
                                 embeddings_metadata=None)

In [56]:
model = tf.keras.models.load_model("model_length_loss3.h5")

In [66]:
model.compile(loss = 'mae', optimizer = 'adam', lr = 0.01)

In [69]:
model.fit(x_train, y_train, batch_size=8, verbose=1, epochs=100,validation_data=(x_val, y_val), callbacks = [reduce_lr])

Train on 17280 samples, validate on 5760 samples
Epoch 1/100
17280/17280 [==============================] - 105s 6ms/sample - loss: 1.4929 - val_loss: 2.8977
Epoch 2/100
17280/17280 [==============================] - 107s 6ms/sample - loss: 1.4733 - val_loss: 3.3755
Epoch 3/100
17280/17280 [==============================] - 107s 6ms/sample - loss: 1.5178 - val_loss: 2.4089
Epoch 4/100
17280/17280 [==============================] - 109s 6ms/sample - loss: 1.4841 - val_loss: 3.1260
Epoch 5/100
17280/17280 [==============================] - 109s 6ms/sample - loss: 1.4711 - val_loss: 2.5095
Epoch 6/100
17280/17280 [==============================] - 108s 6ms/sample - loss: 1.4626 - val_loss: 3.4332
Epoch 7/100
17280/17280 [==============================] - 108s 6ms/sample - loss: 1.4679 - val_loss: 2.7600
Epoch 8/100
17280/17280 [==============================] - 110s 6ms/sample - loss: 1.4800 - val_loss: 2.8556
Epoch 9/100
17280/17280 [==============================] - 109s 6ms/sample - lo

In [70]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_26 (Conv1D)           (None, 2001, 32)          128       
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 667, 32)           0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 667, 32)           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 21344)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               5464320   
_________________________________________________________________
dense_21 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_22 (Dense)             (None, 1)                

In [71]:
model.evaluate(x_test, y_test)

5760/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

2.546284235848321

In [72]:
model.save("model_length_loss3.h5")

In [97]:
for i in range(1, 100):
#     if y_val[i]>=40:
    x_val[i]
    y_val[i]
    testing_x = np.array(x_val[i]).reshape(-1, x_val.shape[1], 1)
    testing_y = np.array(y_val[i]).reshape(-1, y_val.shape[1])
    x = model_revised_2.predict(testing_x).round(1)
    print(f"Predicted {x}, Expected : {testing_y}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[77.3]], Expected : [[75]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Predicted [[90.2]], Expected : [[75]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[141.3]], Expected : [[150]]


In [98]:
model_length.save("model_length.h5")

In [99]:
model_length.fit(x_train, y_train, batch_size=8, verbose=1, epochs=150,validation_data=(x_val, y_val))

Train on 20880 samples, validate on 6960 samples
Epoch 1/150
20880/20880 [==============================] - 218s 10ms/sample - loss: 1.7621 - val_loss: 4.0229
Epoch 2/150
20880/20880 [==============================] - 212s 10ms/sample - loss: 1.7812 - val_loss: 4.6428
Epoch 3/150
20880/20880 [==============================] - 211s 10ms/sample - loss: 1.7842 - val_loss: 4.2326
Epoch 4/150
20880/20880 [==============================] - 214s 10ms/sample - loss: 1.8057 - val_loss: 4.0408
Epoch 5/150
20880/20880 [==============================] - 213s 10ms/sample - loss: 1.8043 - val_loss: 4.0090
Epoch 6/150
20880/20880 [==============================] - 214s 10ms/sample - loss: 1.7932 - val_loss: 4.4932
Epoch 7/150
20880/20880 [==============================] - 213s 10ms/sample - loss: 1.8259 - val_loss: 4.4214
Epoch 8/150
20880/20880 [==============================] - 213s 10ms/sample - loss: 1.8029 - val_loss: 5.2477
Epoch 9/150
20880/20880 [==============================] - 214s 10ms/sa

20880/20880 [==============================] - 172s 8ms/sample - loss: 1.6786 - val_loss: 4.0244
Epoch 148/150
20880/20880 [==============================] - 169s 8ms/sample - loss: 1.6614 - val_loss: 4.4117
Epoch 149/150
20880/20880 [==============================] - 170s 8ms/sample - loss: 1.7126 - val_loss: 4.5391
Epoch 150/150
20880/20880 [==============================] - 171s 8ms/sample - loss: 1.7124 - val_loss: 3.9662


In [45]:
model_length.fit(x_train, y_train, batch_size=8, verbose=1, epochs=50,validation_data=(x_val, y_val))

Train on 21600 samples, validate on 7200 samples
Epoch 1/50
21600/21600 [==============================] - 189s 9ms/sample - loss: 2.1018 - val_loss: 3.7090
Epoch 2/50
21600/21600 [==============================] - 170s 8ms/sample - loss: 2.0884 - val_loss: 5.1294
Epoch 3/50
21600/21600 [==============================] - 176s 8ms/sample - loss: 2.0933 - val_loss: 5.8398
Epoch 4/50
21600/21600 [==============================] - 181s 8ms/sample - loss: 2.0715 - val_loss: 6.2853
Epoch 5/50
21600/21600 [==============================] - 180s 8ms/sample - loss: 2.0635 - val_loss: 4.2928
Epoch 6/50
21600/21600 [==============================] - 182s 8ms/sample - loss: 2.1099 - val_loss: 4.7794
Epoch 7/50
21600/21600 [==============================] - 182s 8ms/sample - loss: 2.0723 - val_loss: 5.4325
Epoch 8/50
21600/21600 [==============================] - 180s 8ms/sample - loss: 2.0610 - val_loss: 5.3328
Epoch 9/50
21600/21600 [==============================] - 181s 8ms/sample - loss: 2.061

KeyboardInterrupt: 

In [43]:
model_length.evaluate(x_test, y_test, verbose=1)
for i in range(1, 100):
#     if y_val[i]>=40:
#     x_test[i]
#     y_test[i]
    testing_x = np.array(x_test[i]).reshape(-1, x_test.shape[1], 1)
    testing_y = np.array(y_test[i]).reshape(-1, y_test.shape[1])
    x = model_length.predict(testing_x).round(1)
    print(f"Predicted {x}, Expected : {testing_y}")

7200/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[40.4]], Expected : [[35]]


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Predicted [[21.]], Expected : [[15]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predicted [[41.1]], Expected : [[40]]


Train on 20880 samples, validate on 6960 samples
Epoch 1/10
20880/20880 [==============================] - 119s 6ms/sample - loss: 24.0779 - val_loss: 18.2474
Epoch 2/10
20880/20880 [==============================] - 94s 4ms/sample - loss: 15.8044 - val_loss: 14.5748
Epoch 3/10
20880/20880 [==============================] - 88s 4ms/sample - loss: 13.4222 - val_loss: 13.7186
Epoch 4/10
20880/20880 [==============================] - 88s 4ms/sample - loss: 10.4051 - val_loss: 10.4971
Epoch 5/10
20880/20880 [==============================] - 88s 4ms/sample - loss: 9.0621 - val_loss: 7.6318
Epoch 6/10
20880/20880 [==============================] - 90s 4ms/sample - loss: 8.4615 - val_loss: 7.8848
Epoch 7/10
20880/20880 [==============================] - 90s 4ms/sample - loss: 8.3068 - val_loss: 8.5396
Epoch 8/10
20880/20880 [==============================] - 89s 4ms/sample - loss: 8.1541 - val_loss: 9.1325
Epoch 9/10
20880/20880 [==============================] - 89s 4ms/sample - loss: 8.068

In [ ]:
model_length1 = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32,kernel_size = 3, activation = 'relu', input_shape = x_train.shape[1:]),
        tf.keras.layers.MaxPooling1D(pool_size=3),
#         tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv1D(128,kernel_size = 3, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(256,kernel_size = 3, activation = 'relu',  kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1)
        ])

In [ ]:
model_length.compile(loss='mae', optimizer='adam')

In [ ]:
model_length.fit(x_train, y_train, batch_size=8, verbose=1, epochs=10,validation_data=(x_val, y_val), callbacks = [reduce_lr  ])

In [25]:
model = tf.keras.models.load_model("to_be_trained.h5")

In [26]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_49 (Conv1D)           (None, 2001, 32)          128       
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 667, 32)           0         
_________________________________________________________________
dropout_73 (Dropout)         (None, 667, 32)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 21344)             0         
_________________________________________________________________
dense_48 (Dense)             (None, 256)               5464320   
_________________________________________________________________
dropout_74 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 128)             

In [27]:
model.compile(loss = 'mae', optimizer = 'adam', lr = 0.1)

In [28]:
model.fit(x_train, y_train, batch_size=8, verbose=1, epochs=100,validation_data=(x_val, y_val), callbacks = [reduce_lr])

Train on 17280 samples, validate on 5760 samples
Epoch 1/100
17280/17280 [==============================] - 115s 7ms/sample - loss: 5.1635 - val_loss: 3.0180
Epoch 2/100
17280/17280 [==============================] - 115s 7ms/sample - loss: 5.1551 - val_loss: 3.9536
Epoch 3/100
17280/17280 [==============================] - 120s 7ms/sample - loss: 5.0605 - val_loss: 3.7813
Epoch 4/100
17280/17280 [==============================] - 115s 7ms/sample - loss: 5.0874 - val_loss: 4.8879
Epoch 5/100
17280/17280 [==============================] - 115s 7ms/sample - loss: 5.0752 - val_loss: 4.6933
Epoch 6/100
17280/17280 [==============================] - 113s 7ms/sample - loss: 5.0240 - val_loss: 6.0374
Epoch 7/100
17280/17280 [==============================] - 113s 7ms/sample - loss: 5.0427 - val_loss: 4.0284
Epoch 8/100
17280/17280 [==============================] - 113s 7ms/sample - loss: 5.1149 - val_loss: 3.5434
Epoch 9/100
17280/17280 [==============================] - 113s 7ms/sample - lo

17280/17280 [==============================] - 92s 5ms/sample - loss: 4.1877 - val_loss: 3.3022
Epoch 70/100
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2560 - val_loss: 3.2990
Epoch 71/100
17280/17280 [==============================] - 93s 5ms/sample - loss: 4.2568 - val_loss: 3.2992
Epoch 72/100
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2507 - val_loss: 3.2970
Epoch 73/100
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2559 - val_loss: 3.2992
Epoch 74/100
17272/17280 [============================>.] - ETA: 0s - loss: 4.2365
Epoch 00074: ReduceLROnPlateau reducing learning rate to 1e-07.
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2363 - val_loss: 3.3026
Epoch 75/100
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2192 - val_loss: 3.3037
Epoch 76/100
17280/17280 [==============================] - 92s 5ms/sample - loss: 4.2859 - val_loss: 3.3036
Epoch 7